In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import sklearn.metrics as metrics
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import KernelPCA

In [2]:
def plot_roc_curve(y_true, y_pred):
    fpr, tpr, threshold = metrics.roc_curve(y_true, y_pred)
    roc_auc = metrics.auc(fpr, tpr)
    plt.figure(figsize=(10, 10))
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.3f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [1, 0], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [3]:
def auc(y_true, y_pred):
    fpr, tpr, threshold = metrics.roc_curve(y_true, y_pred)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [4]:
def lr_na(df):
    X_train = df.copy()
    X_train.dropna(inplace = True)
    df_find = df.loc[:, list(set(df.columns))]
    targets = []
    for i in df_find.columns:
        if len(df_find[df_find[i].isna()]) != 0:
            targets.append(i)            
    for i in targets:
        X_train_target = X_train.loc[:, list(set(X_train.columns) - set(targets))]   
        Y_train_target = X_train[i]
        lr = LinearRegression()
        lr.fit(X_train_target, Y_train_target)
        df_test = df_find[df_find[i].isna()]
        X_test = df_test.loc[:, list(set(df_test.columns) - set(targets))]
        X_test = X_test[X_train_target.columns]
        Y_test = lr.predict(X_test)
        df_find.loc[df_find[i].isna(), i] = Y_test.ravel()
#       targets.remove(i)     
    df = df_find.copy()
    return df

In [5]:
def PLS_na(df):
    X_train = df.copy()
    X_train.dropna(inplace = True)
    df_find = df.loc[:, list(set(df.columns))]
    targets = []
    for i in df_find.columns:
        if len(df_find[df_find[i].isna()]) != 0:
            targets.append(i)  
    current_targets = targets.copy()
    
    for i in targets:
        X_train_target = X_train.loc[:, list(set(X_train.columns) - set(current_targets))]   
        Y_train_target = X_train[i]   
        pls = PLSRegression(n_components = 5)
        pls.fit(X_train_target, Y_train_target)
        df_test = df_find[df_find[i].isna()]
        X_test = df_test.loc[:, list(set(df_test.columns) - set(current_targets))]
        X_test = X_test[X_train_target.columns]
        Y_test = pls.predict(X_test)
        df_find.loc[df_find[i].isna(), i] = Y_test.ravel()
        current_targets.remove(i)     
    df = df_find.copy()
    return df

In [6]:
file_path = 'C:/Users/Frederik/Desktop/TrainSample.csv'
df = pd.read_csv(file_path)
y_train = df["INS"].to_frame()

df1 = df
cat_vars=['BRANCH','RES'] 
for var in cat_vars: 
    cat_list='var'+'_'+var 
    cat_list = pd.get_dummies(df1[var], prefix=var) 
    hr1=df1.join(cat_list) 
    df1=hr1
df = df1.loc[:, list(set(df1.columns) - {'BRANCH','RES', "INS", "id"})].copy()

# df = lr_na(df)
# df = PLS_na(df)

for i in df.columns[df.isnull().any(axis=0)]:
    df[i].fillna(df[i].mean(),inplace=True)

In [7]:
file_path_test = 'C:/Users/Frederik/Desktop/TestSample.csv'

df_test = pd.read_csv(file_path_test)

y_test = df_test["INS"].to_frame()

df1 = df_test
cat_vars=['BRANCH','RES'] 
for var in cat_vars: 
    cat_list='var'+'_'+var 
    cat_list = pd.get_dummies(df1[var], prefix=var) 
    hr1=df1.join(cat_list) 
    df1=hr1
df_test = df1.copy()
df_test = df_test.loc[:, list(set(df_test.columns) - {"BRANCH", "RES", "id", "INS"})]

# df_test = PLS_na(df_test)
# df_test = lr_na(df_test)

for i in df_test.columns[df_test.isnull().any(axis=0)]:
    df_test[i].fillna(df_test[i].mean(),inplace=True)

df_test = df_test[df.columns]

In [8]:
df_bin = pd.DataFrame()
df_bin_test = pd.DataFrame()

df_not_bin = pd.DataFrame()
df_not_bin_test = pd.DataFrame()
for i in df.columns:
    if len(df.loc[df[i] == 0, i]) + len(df.loc[df[i] == 1, i]) == len(df[i]):
        df_bin[i] = df[i]
        df_bin_test[i] = df_test[i]
    else:
        df_not_bin[i] = df[i]
        df_not_bin_test[i] = df_test[i]

In [9]:
# scaler = StandardScaler()  
scaler = MinMaxScaler()
scaler.fit(df_not_bin)

X_train_not_bin = scaler.transform(df_not_bin)
X_test_not_bin = scaler.transform(df_not_bin_test)

poly_reg = PolynomialFeatures(degree = 1)
X_train = poly_reg.fit_transform(X_train_not_bin) 
X_test = poly_reg.transform(X_test_not_bin)

X_train_bin = df_bin.values
X_test_bin = df_bin_test.values

X_train_fin = np.concatenate((X_train, X_train_bin), axis=1)
X_test_fin = np.concatenate((X_test, X_test_bin), axis=1)

In [10]:
pca_model = PCA()
pca_model.fit(X_train_fin)
X_train = pca_model.transform(X_train_fin)
X_test = pca_model.transform(X_test_fin)

best_feats = None
features = list(range(X_train.shape[1]))
Scores = []

while len(features) > 0:
    model = sm.Logit(y_train.values.ravel(), X_train[:, features]).fit()
    L_score = auc(y_train, model.predict(X_train[:, features]))
    Scores.append(L_score)
    if max(Scores) == L_score:
        best_feats = features.copy()
    model = sm.Logit(y_train.values.ravel(), X_train[:, features])
    res = model.fit()
    worse_feature = features[np.argmax(res.pvalues)]
    features.remove(worse_feature)

         Current function value: 0.577826
         Iterations: 35
         Current function value: 0.577826
         Iterations: 35


F:\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
F:\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.577826
         Iterations: 35
         Current function value: 0.577826
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.577826
         Iterations 7


F:\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
F:\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.577826
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.577826
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.577826
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.577829
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.577829
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.577833
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.577833
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.577837
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.577837
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.577846
  

In [11]:
clf = sm.Logit(y_train.values.ravel(), X_train[:, best_feats]).fit(method = "newton")
L_score_train = auc(y_train, clf.predict(X_train[:, best_feats]))
L_score_test = auc(y_test, clf.predict(X_test[:, best_feats]))

print(L_score_train)
print(L_score_test)

Optimization terminated successfully.
         Current function value: 0.577833
         Iterations 7
0.790915359447866
0.7700227686703096


In [21]:
clf.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 5309
Model:                          Logit   Df Residuals:                     5246
Method:                           MLE   Df Model:                           62
Date:                Sun, 17 Apr 2022   Pseudo R-squ.:                  0.1029
Time:                        21:45:16   Log-Likelihood:                -3067.7
converged:                       True   LL-Null:                       -3419.5
Covariance Type:            nonrobust   LLR p-value:                1.621e-109
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.2716      0.051      5.351      0.000       0.172       0.371
x2             0.6776      0.055     12.363      0.000       0.570       0.785
x3             0.2580      0.058      4.477      0.000       0.145       0.371
x4             1.1800      0.066     17.972      0.000       1.051       1.309
x5            -0.1928      0.065     -2.968      0.003      -0.320      -0.065
x6             0.0835      0.067      1.243      0.214      -0.048       0.215
x7             0.2585      0.074      3.483      0.000       0.113       0.404
x8            -0.1522      0.077     -1.972      0.049      -0.303      -0.001
x9             1.0263      0.091     11.291      0.000       0.848       1.204
x10            0.4476      0.092      4.857      0.000       0.267       0.628
x11            0.2435      0.104      2.337      0.019       0.039       0.448
x12           -0.5036      0.104     -4.860      0.000      -0.707      -0.300
x13            0.2615      0.106      2.457      0.014       0.053       0.470
x14           -0.2600      0.106     -2.451      0.014      -0.468      -0.052
x15            0.1549      0.109      1.418      0.156      -0.059       0.369
x16           -0.6892      0.117     -5.914      0.000      -0.918      -0.461
x17            0.4234      0.115      3.679      0.000       0.198       0.649
x18            0.0615      0.129      0.477      0.633      -0.191       0.314
x19            0.9029      0.146      6.204      0.000       0.618       1.188
x20           -0.7562      0.146     -5.163      0.000      -1.043      -0.469
x21           -0.6337      0.149     -4.247      0.000      -0.926      -0.341
x22            0.0864      0.147      0.588      0.557      -0.202       0.375
x23            0.1330      0.148      0.897      0.370      -0.158       0.424
x24            0.4072      0.150      2.714      0.007       0.113       0.701
x25            0.2986      0.163      1.828      0.068      -0.022       0.619
x26           -0.7887      0.183     -4.321      0.000      -1.146      -0.431
x27           -0.0567      0.176     -0.322      0.747      -0.402       0.288
x28           -0.2050      0.178     -1.149      0.251      -0.555       0.145
x29            0.3412      0.204      1.675      0.094      -0.058       0.740
x30           -0.1567      0.200     -0.785      0.433      -0.548       0.235
x31            0.2776      0.198      1.399      0.162      -0.111       0.666
x32           -0.2497      0.216     -1.158      0.247      -0.672       0.173
x33           -0.1145      0.225     -0.510      0.610      -0.555       0.326
x34           -0.0809      0.240     -0.338      0.736      -0.551       0.389
x35            0.7824      0.270      2.901      0.004       0.254       1.311
x36           -0.4007      0.280     -1.430      0.153      -0.950       0.149
x37            1.4605      0.296      4.926      0.000       0.879       2.042
x38            0.7564      0.309      2.445      0.014       0.150       1.363
x3